In [1]:
# PSO
# swarm = [Particle...]
# while not stop_condition():
#     for particle in swarm:
#         particle.calc_velocity - c_i*inercija + r_l*c_l*licno + r_j*c_j*jato
#         particle.calc_position - trenutna pozicija + brzina
# resenje = jato

In [2]:
import numpy as np
import random

In [3]:
def rastrigin(x):
    A = 10
    n = len(x)
    return A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n))

In [4]:
class Particle:
    global_best_position = None
    global_best_value = None
    
    def __init__(self, bounds, obj_func, c_i, c_p, c_g):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))

        self.personal_best_position = self.position.copy()
        
        self.obj_func = obj_func
        self.c_i = c_i
        self.c_p = c_p
        self.c_g = c_g
        
        self.value = obj_func(self.position)
        self.personal_best_value = obj_func(self.personal_best_position)

        if Particle.global_best_value is None or Particle.global_best_value > self.value:
            Particle.global_best_value = self.value
            Particle.global_best_position = self.position.copy()
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.value = self.obj_func(self.position)
        if self.value < self.personal_best_value:
            self.personal_best_value = self.value
            self.personal_best_position = self.position.copy()
            if self.value < Particle.global_best_value:
                Particle.global_best_value = self.value
                Particle.global_best_position = self.position.copy()
    
    def update_velocity(self):
        r_p = np.random.random(self.velocity.shape)
        r_s = np.random.random(self.velocity.shape)
# mozda bi imalo smisla da normiramo vektore pre ovoga
# mozda i value ukljuciti pazljivo
        cognitive_velocity = r_p * self.c_p * (self.personal_best_position - self.position)
        social_velocity = r_s * self.c_g * (Particle.global_best_position - self.position)
        inertia = self.c_i * self.velocity
        self.velocity = inertia + cognitive_velocity + social_velocity

update_velocity:
$$vel_{cog} = r_p * c_p * (pbp - pos)$$
$$vel_{soc} = r_s * c_g * (gbp - pos)$$
$$inertia = c_i * velocity$$
$$velocity = inertia + vel_{cog} + vel_{soc}$$

In [5]:
def pso(swarm_size, num_dimensions, c_i, c_p, c_g, num_iters):
    bounds = [(-5.12, 5.12) for _ in range(num_dimensions)]
    
    swarm = [Particle(bounds=bounds,
                      obj_func=rosenbrock,
                      c_i=c_i,
                      c_p=c_p,
                      c_g=c_g) for _ in range(swarm_size)]
    for _ in range(num_iters):
        for particle in swarm:
            particle.update_velocity()
            particle.update_position()

    print(Particle.global_best_position)
    print(Particle.global_best_value)
    
    Particle.global_best_position = None
    Particle.global_best_value = None

In [6]:
def rosenbrock(x):
    a = 1
    b = 100
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2

In [7]:
pso(swarm_size=50, num_dimensions=2, c_i=0.7, c_p=1.0, c_g=1.0, num_iters=100)

[1.00596696 1.0117931 ]
3.871688088035338e-05
